In [1]:
# NOTE: this is a custom cell that contains the common imports I personally 
# use these may/may not be necessary for the following examples

# DL framework
import tensorflow as tf

from datetime import datetime

# common packages
import numpy as np
import os # handling file i/o
import sys
import math
import time # timing epochs
import random

# for ordered dict when building layer components
import collections

# plotting pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import pyplot
from matplotlib import colors # making colors consistent
from mpl_toolkits.axes_grid1 import make_axes_locatable # colorbar helper


# from imageio import imread # read image from disk
# + data augmentation
from scipy import ndimage
from scipy import misc


import pickle # manually saving best params
from sklearn.utils import shuffle # shuffling data batches
from tqdm import tqdm # display training progress bar

# const
SEED = 42

# Helper to make the output consistent
def reset_graph(seed=SEED):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# helper to create dirs if they don't already exist
def maybe_create_dir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        print("{} created".format(dir_path))
    else:
        print("{} already exists".format(dir_path))
    
def make_standard_dirs(saver=True, best_params=True, tf_logs=True):
    # `saver/` will hold tf saver files
    maybe_create_dir("saver")
    # `best_params/` will hold a serialized version of the best params
    # I like to keep this as a backup in case I run into issues with
    # the saver files
    maybe_create_dir("best_params")
    # `tf_logs/` will hold the logs that will be visable in tensorboard
    maybe_create_dir("tf_logs")

    
# set tf log level to supress messages, unless an error
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Important Version information
print("Python: {}".format(sys.version_info[:]))
print('TensorFlow: {}'.format(tf.__version__))

# Check if using GPU
if not tf.test.gpu_device_name():
    print('No GPU')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
    
reset_graph()

Python: (3, 5, 4, 'final', 0)
TensorFlow: 1.6.0-dev20180105
No GPU


In [2]:
make_standard_dirs()

saver created
best_params created
tf_logs created


In [3]:
# these two functions (get_model_params and restore_model_params) are 
# ad[a|o]pted from; 
# https://github.com/ageron/handson-ml/blob/master/11_deep_learning.ipynb
def get_model_params():
    global_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    return {global_vars.op.name: value for global_vars, value in 
            zip(global_vars, tf.get_default_session().run(global_vars))}

def restore_model_params(model_params, g, sess):
    gvar_names = list(model_params.keys())
    assign_ops = {gvar_name: g.get_operation_by_name(gvar_name + "/Assign")
                  for gvar_name in gvar_names}
    init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
    feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
    sess.run(assign_ops, feed_dict=feed_dict)

# these two functions are used to manually save the best
# model params to disk
def save_obj(obj, name):
    with open('best_params/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open('best_params/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

## Dataset

In [4]:
ROOT_DATA = "../../ROOT_DATA/"
DATA_DIR = "mnist_data"

MNIST_TRAINING_PATH = os.path.join(ROOT_DATA, DATA_DIR)
# ensure we have the correct directory
for _, _, files in os.walk(MNIST_TRAINING_PATH):
    files = sorted(files)
    for filename in files:
        print(filename)

t10k-images-idx3-ubyte.gz
t10k-labels-idx1-ubyte.gz
train-images-idx3-ubyte.gz
train-labels-idx1-ubyte.gz


In [5]:
from tensorflow.examples.tutorials.mnist import input_data
MNIST = input_data.read_data_sets(MNIST_TRAINING_PATH, one_hot=True)

Extracting ../../ROOT_DATA/mnist_data/train-images-idx3-ubyte.gz
Extracting ../../ROOT_DATA/mnist_data/train-labels-idx1-ubyte.gz
Extracting ../../ROOT_DATA/mnist_data/t10k-images-idx3-ubyte.gz
Extracting ../../ROOT_DATA/mnist_data/t10k-labels-idx1-ubyte.gz


In [16]:
def create_hyper_params():
    data_params = {}
    data_params['n_epochs'] = 150
    data_params['batch_size'] = 64
    data_params['buffer_size'] = 128 # for shuffling

    data_params['init_lr'] = 1e-5

    
    return data_params

In [43]:
def build_graph(data_params):
    g = tf.Graph()
    n_outputs = 10
    IMG_HEIGHT = 28
    IMG_WIDTH = 28
    CHANNELS = 1
    with g.as_default():
        with tf.name_scope("inputs"):
            X = tf.placeholder(tf.float32, shape=(None, 784), name="data") # Input
            X_reshaped = tf.reshape(X, shape=[-1, IMG_HEIGHT, IMG_WIDTH, CHANNELS])
            y = tf.placeholder(tf.float32, shape=(None, n_outputs), name="labels") # Target

        with tf.name_scope("cnn"):
            h_1 = tf.layers.conv2d(X_reshaped, filters=12, kernel_size=3, 
                                   padding='SAME', strides=1, name="conv_1")
            h_2 = tf.layers.conv2d(h_1, filters=24, kernel_size=3, 
                                   padding='SAME', strides=1, name="conv_2")
            h_3 = tf.layers.conv2d(h_1, filters=36, kernel_size=3, 
                                   padding='SAME', strides=2, name="conv_3")
            h_4 = tf.layers.max_pooling2d(h_3, pool_size=[2,2], 
                                          strides=2, name="max_pool_01")
            last_shape = int(np.prod(h_4.get_shape()[1:]))
            h_4_flat = tf.reshape(h_4, shape=[-1, last_shape])
            h_5 = tf.layers.dense(h_4_flat, 64, name="layer_05")
            logits = tf.layers.dense(h_5, n_outputs, name="logits")

        with tf.name_scope("loss"):
            xentropy = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits)
            batch_loss = tf.reduce_mean(xentropy, name="loss")
        
        with tf.name_scope("train"):
            optimizer = tf.train.GradientDescentOptimizer(data_params['init_lr'])
            training_op = optimizer.minimize(batch_loss)
            
        with tf.name_scope("save_session"):
            init_global = tf.global_variables_initializer()
            init_local = tf.local_variables_initializer()
            saver = tf.train.Saver()
            
        # Ops: training metrics
        with tf.name_scope("metrics"):
            # ================================== performance
            with tf.name_scope("common"):
                preds = tf.nn.softmax(logits, name="prediction")
                y_true_cls = tf.argmax(y,1)
                y_pred_cls = tf.argmax(preds,1)
                correct_prediction = tf.equal(y_pred_cls, y_true_cls, name="correct_predictions")
                batch_acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
            with tf.name_scope("train_metrics") as scope:
                train_auc, train_auc_update = tf.metrics.auc(labels=y, predictions=preds)
                train_acc, train_acc_update = tf.metrics.accuracy(labels=y_true_cls, predictions=y_pred_cls)
                train_acc_vars = tf.contrib.framework.get_variables(scope, collection=tf.GraphKeys.LOCAL_VARIABLES)
                train_met_reset_op = tf.variables_initializer(train_acc_vars, name="train_met_reset_op")
            with tf.name_scope("val_metrics") as scope:
                val_auc, val_auc_update = tf.metrics.auc(labels=y, predictions=preds)
                val_acc, val_acc_update = tf.metrics.accuracy(labels=y_true_cls, predictions=y_pred_cls)
                val_acc_vars = tf.contrib.framework.get_variables(scope, collection=tf.GraphKeys.LOCAL_VARIABLES)
                val_met_reset_op = tf.variables_initializer(val_acc_vars, name="val_met_reset_op")
            with tf.name_scope("test_metrics") as scope:    
                test_acc, test_acc_update = tf.metrics.accuracy(labels=y_true_cls, predictions=y_pred_cls)
                test_acc_vars = tf.contrib.framework.get_variables(scope, collection=tf.GraphKeys.LOCAL_VARIABLES)
                test_acc_reset_op = tf.variables_initializer(test_acc_vars, name="test_met_reset_op")

            # =============================================== loss 
            with tf.name_scope("train_loss_eval") as scope:
                train_mean_loss, train_mean_loss_update = tf.metrics.mean(batch_loss)
                train_loss_vars = tf.contrib.framework.get_variables(scope, collection=tf.GraphKeys.LOCAL_VARIABLES)
                train_loss_reset_op = tf.variables_initializer(train_loss_vars, name="train_loss_reset_op")
            with tf.name_scope("val_loss_eval") as scope:
                val_mean_loss, val_mean_loss_update = tf.metrics.mean(batch_loss)
                val_loss_vars = tf.contrib.framework.get_variables(scope, collection=tf.GraphKeys.LOCAL_VARIABLES)
                val_loss_reset_op = tf.variables_initializer(val_loss_vars, name="val_loss_reset_op")
            with tf.name_scope("test_loss_eval")as scope:
                test_mean_loss, test_mean_loss_update = tf.metrics.mean(batch_loss)
                test_loss_vars = tf.contrib.framework.get_variables(scope, collection=tf.GraphKeys.LOCAL_VARIABLES)
                test_loss_reset_op = tf.variables_initializer(test_loss_vars, name="test_loss_rest_op")

        # --- create collections
        for node in (saver, init_global, init_local):
            g.add_to_collection("save_init", node)
        for node in (X, y, training_op):
            g.add_to_collection("main_ops", node)
        for node in (preds, y_true_cls, y_pred_cls):
            g.add_to_collection("preds", node)
        for node in (train_auc, train_auc_update, train_acc, train_acc_update, train_met_reset_op):
            g.add_to_collection("train_metrics", node)
        for node in (val_auc, val_auc_update, val_acc, val_acc_update, val_met_reset_op):
            g.add_to_collection("val_metrics", node)
        for node in (test_acc, test_acc_update, test_acc_reset_op):
            g.add_to_collection("test_metrics", node)
        for node in (train_mean_loss, train_mean_loss_update, train_loss_reset_op):
            g.add_to_collection("train_loss", node)
        for node in (val_mean_loss, val_mean_loss_update, val_loss_reset_op):
            g.add_to_collection("val_loss", node)
        for node in (test_mean_loss, test_mean_loss_update, test_loss_reset_op):
            g.add_to_collection("test_loss", node)
            
        # ===================================== tensorboard
        with tf.name_scope("tensorboard_writer") as scope:
            epoch_train_loss_scalar = tf.summary.scalar('train_epoch_loss', train_mean_loss)
            epoch_train_acc_scalar = tf.summary.scalar('train_epoch_acc', train_acc)
            epoch_train_auc_scalar = tf.summary.scalar('train_epoch_auc', train_auc)
            epoch_train_write_op = tf.summary.merge([epoch_train_loss_scalar, epoch_train_acc_scalar, epoch_train_auc_scalar], name="epoch_train_write_op")

            # ===== epoch, validation
            epoch_validation_loss_scalar = tf.summary.scalar('validation_epoch_loss', val_mean_loss)
            epoch_validation_acc_scalar = tf.summary.scalar('validation_epoch_acc', val_acc)
            epoch_validation_auc_scalar = tf.summary.scalar('validation_epoch_auc', val_auc)
            epoch_validation_write_op = tf.summary.merge([epoch_validation_loss_scalar, epoch_validation_acc_scalar, epoch_validation_auc_scalar], name="epoch_validation_write_op")
        
        for node in (epoch_train_write_op, epoch_validation_write_op):
            g.add_to_collection("tensorboard", node)
    
    return g

In [44]:
def train_graph(g):
    saver, init_global, init_local = g.get_collection("save_init")
    X, y, training_op = g.get_collection("main_ops")
    preds, y_true_cls, y_pred_cls = g.get_collection("preds")
    train_auc, train_auc_update, train_acc, train_acc_update, train_met_reset_op = g.get_collection("train_metrics")
    val_auc, val_auc_update, val_acc, val_acc_update, val_met_reset_op = g.get_collection("val_metrics")
    #test_auc, test_auc_update, test_acc, test_acc_update, test_acc_reset_op = g.get_collection("test_metrics")
    train_mean_loss, train_mean_loss_update, train_loss_reset_op = g.get_collection("train_loss")
    val_mean_loss, val_mean_loss_update, val_loss_reset_op = g.get_collection("val_loss")
    #test_mean_loss, test_mean_loss_update, test_loss_reset_op = g.get_collection("test_loss")
    epoch_train_write_op, epoch_validation_write_op = g.get_collection("tensorboard")

    train_writer = tf.summary.FileWriter(os.path.join("tf_logs","train"))
    val_writer = tf.summary.FileWriter(os.path.join("tf_logs","validation"))
    
    with tf.Session(graph=g) as sess:
        sess.run([init_global, init_local])
        
        for e in tqdm(range(1,data_params['n_epochs']+1)):
            sess.run([val_met_reset_op,val_loss_reset_op,train_met_reset_op,train_loss_reset_op])
            
            n_batches = int(MNIST.train.num_examples/data_params['batch_size'])
            for i in range(1, n_batches+1):
                data, target = MNIST.train.next_batch(data_params['batch_size'])
                sess.run([training_op, train_auc_update, train_acc_update, train_mean_loss_update], feed_dict={X:data, y:target})
        
        # write average for epoch
        summary = sess.run(epoch_train_write_op)    
        train_writer.add_summary(summary, e)
        train_writer.flush()
        
        # run validation
        n_batches = int(MNIST.validation.num_examples/data_params['batch_size'])
        for i in range(1,n_batches+1):
            Xb, yb = MNIST.validation.next_batch(data_params['batch_size'])
            sess.run([val_auc_update, val_acc_update, val_mean_loss_update], feed_dict={X:data, y:target})
        
        summary = backprop_sess.run(epoch_validation_write_op) 
        val_writer.add_summary(summary, e)
        val_writer.flush()
        
    train_writer.close()
    val_writer.close()
    return sess

In [45]:
reset_graph()
data_params = create_hyper_params()
g = build_graph(data_params)
sess = train_graph(g)

  1%|          | 1/150 [00:25<1:03:17, 25.49s/it]

KeyboardInterrupt: 